# Introducing The Colonel Blotto Problem

We will be solving a sub-game of the more general [Colonel Blotto](https://en.wikipedia.org/wiki/Blotto_game) resource allocation problem. In this game, two millitary generals must allocate S soldiers across N possible battlefields. Allocating more soldiers to a battlefield than one's opponent results in a conquest. Allocating an equal number of soldiers results in a tie. The player with the most conquests wins the round.

All code is available on my [CFRM repository](https://github.com/pranavwalia/CFRM/tree/master/Colonel%20Blotto).

## Table of Contents


## Colonel Blotto Toy Game (S, N) = (5, 3)
Write a program that solves the Colonel Blotto problem for S = 5 soldiers and N = 3 battlefields.

## Mixed vs Pure Strategy
In the context of this game, a 'pure' strategy is a single possible allocation of soldiers eg. (3,1,1). A mixed strategy is a mixed allocation of soldiers. Example: choosing (3,1,1) 30% of the time and (5,00) 70% of the time. We will begin by implementing a function that can generate all possible pure strategies. 

A valid allocation can be modeled as a linear equation consisting of N variables where the sum of those variables are S
$$
\sum_{i = 1}^{N} X_{i} = S
$$
In the case of this game, N = 3 and S = 5
$$
\sum_{i = 1}^{3} X_{i} = 5
$$
Where $$X_{i}$$ is an integer and $$0 \leq X_{i} \leq 5$$

Knowing this, we will write a function that generates all possible solutions to the equation under these constraints. Each solution will be a possible pure strategy.

In [1]:
import random
import plotly.express as pt
import pandas as pd

### Generate Strategies
Since our problem only contains 3 battle-fields, we can straightforwardly generate all solutions to the equation using 2 nested loops

In [2]:
#Generate all pure strategies for n = 3, S = 5
def generateStrategies():
    s = 5
    strats = []
    for x1 in range(s + 1):
        current_strat = []
        for x2 in range((s + 1) - x1):
            for x3 in range((s + 1) - (x1 + x2)):
                if (x1 + x2 + x3) == s:
                    current_strat = [x1,x2,x3]
                    strats.append(current_strat)
    return strats        

Running the function will generate 21 possible unique allocations. 

In [3]:
generateStrategies()

[[0, 0, 5],
 [0, 1, 4],
 [0, 2, 3],
 [0, 3, 2],
 [0, 4, 1],
 [0, 5, 0],
 [1, 0, 4],
 [1, 1, 3],
 [1, 2, 2],
 [1, 3, 1],
 [1, 4, 0],
 [2, 0, 3],
 [2, 1, 2],
 [2, 2, 1],
 [2, 3, 0],
 [3, 0, 2],
 [3, 1, 1],
 [3, 2, 0],
 [4, 0, 1],
 [4, 1, 0],
 [5, 0, 0]]

## Compute Utility
Computing utility for a round of Colonel Blotto is as simple as counting how many battle-fields are won in a single round. The arguments we pass to the function will be lists of length 3 representing strategy profiles.

In [4]:
#Compute utility for player 1
#Assume p1 and p2 are 
def getUtility(p1,p2):
    u = 0
    for i in range(len(p1)):
        if p1[i] > p2[i]:
            u+=1
    return u

## Get Action According To Current Strategy
Next, we will want to create a function that randomly allocates soldiers to battle fields based on a given strategy profile. A strategy profile can be represented as a list of 21 real numbers $$X$$
where $$ 0 \leq X_{i}\leq 1$$
Each element of the list represents a frequency at which a corresponding allocation is chosen. As such, the sum of all the elements in the list must be equal to 1.
$$
\sum_{i = 1}^{21}X_{i} = 1
$$

Let us first make a function that generates a default strategy profile where each action is equally weighted.

In [5]:
#This function generates a default strategy where each action profile has equal weighting
def defaultStrat():
    actionProfiles = generateStrategies()
    strat = []
    for i in range(len(actionProfiles)):
        strat.append(1/len(actionProfiles))
    return strat

In [6]:
defaultStrat()

[0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616,
 0.047619047619047616]

Now we will write a function that randomly chooses an action profile given a strategy

In [7]:
#[List] -> [X1,X2,X3]
def getAction(strategy):
    rand = random.random()
    actions = generateStrategies()
    leftSum = 0
    rightSum = 0
    for i in range(len(strategy)):
        rightSum+=strategy[i]
        if rand > leftSum and rand <= rightSum:
            return actions[i]
        else:
            leftSum+=strategy[i]
    return actions[0]

In [8]:
getAction(defaultStrat())

[5, 0, 0]

## Utility of One Strategy vs Another
Additionally, we want a way of measuring the utility of one strategy vs another. This will come in handy when evaluating the results of our algorithm.

* Let $A$ represent the set of all action profiles where $A_{1},A_{2}..A_{21}$ are the 21 possible action profiles in the game 

* Let $G$ be the strategy profile (list of frequencies for player1's action selection

* Let $R$ be the strategy profile (list of frequencies for player1's action selection

* Let $\alpha(A_{x},A_{i})$ be the utility of action $A_{x}$ against action $A_{i}$

* Let $U(A_{x},R)$ be the utility function returning the average utility of playing action profile $A_{x}$ against strategy profile $S$

$$
U(A_{x},R) = \sum_{i = 1}^{21}\alpha(A_{x},A_{i})R_{i}
$$

* Let $U_{A}(G,R)$ be the average utility of strategy $G$ this is calculated by multiplying the utility of every action in $A$ by the frequency it is chosen in $G$.

$$
U_{A}(G,R) = \sum_{i = 1}^{21} U(A_{i},R)G_{i}
$$

In [9]:
def utilityAverage(g,r):
    actions = generateStrategies()
    actionUtilities = [0] * 21
    for i in range(len(actions)):
        u = 0
        for j in range(len(actions)):
            u+=((getUtility(actions[i],actions[j]) * r[j]))
        actionUtilities[i] = u * g[i]
    return sum(actionUtilities)

## Get Regret Matched Strategy
We will now write a function that computes a regret matched strategy based on a regret sum and a strategy sum. The regret sum is a list of 21 elements where each element represents the regret differential of each action in a particular round of the game. The strategySum is the current accumulated sum of all normalized strategies.

Our function will return a tuple of the most recent strategy and the sum of all previous strategies in the format

$$
(strategy,strategySum)
$$

### Regret Sum
The regretSum is calculated by taking the difference of the expected value of each alternate action and the action that was used in a particular round. Positive regrets are ones where an agent could have gained utility by choosing an alternate action profile. Negative regrets are actions that would have been worse than the choosen strategy.

### Strategy Sum
The strategy sum is the sum of all previous normalized regret matched strategies.

### Why we normalize
The regret of each decision must be matched in terms of the total regret generated by choosing a particular action in a round rather than the total absolute regret. This ensures that strategic adjustments are not skewed by variance resulting in an incorrect average strategy profile.

In [10]:
##Returns the adjusted strategy after an iteration
def getStrategy(regretSum,strategySum):
    actions = 21
    normalizingSum = 0
    strategy = [0] * 21
    #Normalizingsum is the sum of positive regrets. 
    #This ensures do not 'over-adjust' our strategy
    for i in range(0,actions):
        if regretSum[i] > 0:
            strategy[i] = regretSum[i]
        else:
            strategy[i] = 0
        normalizingSum += strategy[i]
    ##This loop normalizes our updated strategy
    for i in range(0,actions):
        if normalizingSum > 0:
            strategy[i] = strategy[i]/normalizingSum
        else:
            #Default to 33%
            strategy[i] = 1.0 / actions
        strategySum[i] += strategy[i]
    return (strategy,strategySum)

## Training Algorithm For Maximally Exploitative Strategy
We will write a training algorithm that calculates a maximally exploitative strategy for player1 against a static strategy from player 2. The function will simulate a set number of iterations. At each iteration, it will retrieve a regret-matched strategy for player1. Using this strategy, it will select an action. Afterwards, the static action profile will be used to retrieve an action for player 2. Finally, regrets will be calculated and player1's strategy will be adjusted.

In [11]:
def train(iterations,regretSum,oppStrategy):
    actionUtility = [0] * 21
    strategySum = [0] * 21
    actions = 21
    for i in range(0,iterations):
        ##Retrieve Actions
        t = getStrategy(regretSum,strategySum)
        strategy = t[0]
        strategySum = t[1]
        myAction = getAction(strategy)
        oppAction = getAction(oppStrategy)   
        actionList = generateStrategies()
        
        for i in range(len(actionList)):
            actionUtility[i] = getUtility(actionList[i],oppAction)
            
        #Add the regrets from this decision
        for i in range(actions):
            regretSum[i] += actionUtility[i] - getUtility(myAction,oppAction)
    return strategySum

### Retrieve Maximally Exploitative Strategy

In [12]:
def getMaxExploitStrategy(iterations,oppStrategy):
    regretSum = [0] * 21
    oppStrategy = defaultStrat()
    iterations = 1000000
    strategySum = train(iterations,regretSum,oppStrategy)
    normalizingSum = 0
    actions = 21
    avgStrategy = [0] * 21
    for i in range(0,actions):
        normalizingSum += strategySum[i]
    for i in range(0,actions):
        if normalizingSum > 0:
            avgStrategy[i] = strategySum[i] / normalizingSum
        else:
            avgStrategy[i] = 1.0 / actions
    return avgStrategy

### Maximally Exploitative Strategy vs Default
Using the training algorithm we will observe the adjustment to the default strategy of choosing every action profile with an equal weighting.

In [13]:
maxExpl1 = getMaxExploitStrategy(10000000,defaultStrat())

The utility of the maximally exploitative strategy will be higher than the default

In [14]:
utilityAverage(defaultStrat(),defaultStrat())

1.1904761904761907

In [15]:
utilityAverage(maxExpl1,defaultStrat())

1.3333189648370247

#### Analyzing Maximally Exploitative Strategy vs Default
Let's convert our results into a convinient format for analysis

In [16]:
def stratTodf(strategy):
    actions = generateStrategies()
    act_to_string = []
    for i in actions:
        act_to_string.append("".join(str(i)))
    return pd.DataFrame(strategy,act_to_string)

In [17]:
df = stratTodf(maxExpl1)
df.head()

,0
"[0, 0, 5]",9.523810e-08
"[0, 1, 4]",2.387788e-07
"[0, 2, 3]",9.523810e-08
"[0, 3, 2]",9.523810e-08
"[0, 4, 1]",9.523810e-08


In [25]:
plot1 = pt.bar(df,title = "Strategy vs Frequency")
plot1.show()

The maximally exploitative strategy against the equally weighted default strategy involves some permutation of a [1,2,2] strategy.

### Maximally Exploitative Strategy vs [0,0,5]
Now we will observe the maximally exploitative strategy against a pure strategy of assigning 5 soldiers to the third battlefield.

In [19]:
zero_zero_five = [1] + [0] * 20
maxExpl2 = getMaxExploitStrategy(10000000,zero_zero_five)

Once again, the maximally exploitative strategy will have a higher utility than the default strategy

In [20]:
utilityAverage(defaultStrat(),zero_zero_five)

1.4285714285714286

In [21]:
utilityAverage(maxExpl2,zero_zero_five)

1.9999921276638861

In [22]:
df2 = stratTodf(maxExpl2)

In [26]:
pt.bar(df2,title = "Strategy vs Frequency")

## The Optimal Solution
Now let's write a training algorithm for the optimal solution. This time we'll have both agents adjust their strategies.

In [52]:
#Two player training Function
def train2Player(iterations,regretSum1,regretSum2,p2Strat):
    ##Adapt Train Function for two players
    actionList = generateStrategies()
    actions = 21
    actionUtility1 = [0] * 21
    actionUtility2 = [0] * 21
    strategySum1 = [0] * 21
    strategySum2 = [0] * 21
    for i in range(0,iterations):
        ##Retrieve Actions
        t1 = getStrategy(regretSum1,strategySum1)
        strategy1 = t1[0]
        strategySum1 = t1[1]
        myaction = getAction(strategy1)
        
        t2 = getStrategy(regretSum2,p2Strat)
        strategy2 = t2[0]
        strategySum2 = t2[1]
        otherAction = getAction(strategy2)
        
        for i in range(actions):
            actionUtility1[i] = getUtility(actionList[i],otherAction)
            actionUtility2[i] = getUtility(actionList[i],myaction)
    
        for i in range(actions):
            regretSum1[i] += actionUtility1[i] - getUtility(actionList[i],otherAction)
            regretSum2[i] += actionUtility2[i] - getUtility(actionList[i],myaction)
        
    return (strategySum1, strategySum2)

#Returns a nash equilibrium reached by two opponents through CFRM
def Colonel_Blotto_Nash(iterations,oppStrat):
    strats = train2Player(iterations,[0] * 21,[0] * 21,oppStrat)
    s1 = sum(strats[0])
    s2 = sum(strats[1])
    for i in range(21):
        if s1 > 0:
            strats[0][i] = strats[0][i]/s1
        if s2 > 0:
            strats[1][i] = strats[1][i]/s2
    return strats

In [58]:
nashStrat = Colonel_Blotto_Nash(1000000,[1] + [0] * 20)

The utility of both strategies will now be roughly equal

In [59]:
utilityAverage(nashStrat[0],nashStrat[1])

1.1904764285711906

In [60]:
utilityAverage(nashStrat[1],nashStrat[0])

1.1904759523811914

In [61]:
dfp1 = stratTodf(nashStrat[0])
dfp2 = stratTodf(nashStrat[1])

In [62]:
pt.bar(dfp1).show()
pt.bar(dfp2).show()

# Sources and Further Reading
This problem was originally posed as an exercise in [Introduction To Counterfactual Regret Minimization, Todd W. Neller and Lanctot](http://modelai.gettysburg.edu/2013/cfr/cfr.pdf). I could not find an available solution to this problem online so I took it upon myself to make one.